In [ ]:
word_embedding = self.mllm_encoder.embeddings(word_tokens.to(self.device))
        
        prompt_enhance_embeddings = self.learnable_embeddings(
            torch.arange(self.seq_max).to(self.device)).unsqueeze(0).repeat(
            [len(texts), 1, 1])
        prompt_enhance_tokens = torch.zeros(len(texts), 77).to(self.device)

        for i in range(len(texts)):
            ind = torch.argmax(word_tokens[i], -1)
            prompt_enhance_embeddings[i, 0] = word_embedding[i, 0]
            prompt_enhance_embeddings[i, self.learnable_prompt_len + 1: self.learnable_prompt_len + ind] = word_embedding[i, 1: ind]
            prompt_enhance_embeddings[i, self.learnable_prompt_len + ind + self.learnable_prompt_len] = word_embedding[i, ind]
            attention_mask[i, : self.learnable_prompt_len * 2 + ind + 1] = 1
            prompt_enhance_tokens[i, self.learnable_prompt_len * 2 + ind] = word_tokens[i, ind]

In [6]:
from transformers import (
    DebertaV2Tokenizer, DebertaV2Model,
    AutoTokenizer, AutoModel
)
import torch
import torch.nn as nn

In [8]:
tokenizer = DebertaV2Tokenizer.from_pretrained("/export/space0/qiu-y/model/deberta-v3-base")
seq_max = 77
texts = [
        'normal', 'abuse', 'arrest', 'arson', 'assault',
        'burglary', 'explosion', 'fighting', 'roadAccidents',
        'robbery', 'shooting', 'shoplifting', 'stealing', 'vandalism']

encoded = tokenizer(
            texts,
            max_length=seq_max,
            padding='max_length',
            truncation=True,
            return_tensors='pt',
        )

word_tokens = encoded["input_ids"]
attention_mask = encoded["attention_mask"]


In [ ]:
learnable_prompt_len = 10
prompt_hidden_dim = 512

word_embedding = torch.randn((14, 77, 512))
learnable_embeddings = nn.Embedding(seq_max, prompt_hidden_dim) # [77, 512]
prompt_enhance_embeddings = learnable_embeddings(torch.arange(seq_max)).unsqueeze(0).repeat([len(texts), 1, 1])  # [14, 77, 512]
prompt_enhance_tokens = torch.zeros(len(texts), 77) # [14, 77]

for i in range(len(texts)):
    ind = torch.argmax(word_tokens[i], -1)
    # print(prompt_enhance_embeddings[i, 0])
    print(word_embedding[i, 0].shape)
    prompt_enhance_embeddings[i, 0] = word_embedding[i, 0]  # 复制word_embedding中第一个向量到 prompt_enhance_embeddings的第一个位置
    prompt_enhance_embeddings[i, learnable_prompt_len + 1: learnable_prompt_len + ind] = word_embedding[i, 1: ind]
    prompt_enhance_embeddings[i, learnable_prompt_len + ind + learnable_prompt_len] = word_embedding[i, ind]
    attention_mask[i, : learnable_prompt_len * 2 + ind + 1] = 1
    prompt_enhance_tokens[i, learnable_prompt_len * 2 + ind] = word_tokens[i, ind]





torch.Size([512])
torch.Size([512])
torch.Size([512])
torch.Size([512])
torch.Size([512])
torch.Size([512])
torch.Size([512])
torch.Size([512])
torch.Size([512])
torch.Size([512])
torch.Size([512])
torch.Size([512])
torch.Size([512])
torch.Size([512])


In [13]:
from transformers import DebertaV2Tokenizer

# 使用您自己的模型路径
vid_mllm_ckpt = "/export/space0/qiu-y/model/deberta-v3-base" 
tokenizer = DebertaV2Tokenizer.from_pretrained(vid_mllm_ckpt)

# --- 验证部分 ---

# 1. 直接查看分词器中定义的[CLS] token是什么
print(f"分词器定义的[CLS] token是: {tokenizer.cls_token}")
print(f"它的ID是: {tokenizer.cls_token_id}")

print("-" * 30)

# 2. 观察分词器如何处理一段普通文本
text = "hello world"
encoded = tokenizer(text)

print(f"原始文本: '{text}'")
print(f"编码后的Token ID: {encoded['input_ids']}")

# 3. 将ID解码回Token，看得更清楚
tokens = tokenizer.convert_ids_to_tokens(encoded['input_ids'])
print(f"解码回Token: {tokens}")

分词器定义的[CLS] token是: [CLS]
它的ID是: 1
------------------------------
原始文本: 'hello world'
编码后的Token ID: [1, 12018, 447, 2]
解码回Token: ['[CLS]', '▁hello', '▁world', '[SEP]']
